# Prédiction des Notes au Piano


Ce projet utilise l'apprentissage profond pour prédire des notes de piano à partir de fichiers audio. Les étapes incluent :

1. Préparation des données
2. Entraînement d'un modèle CNN
3. Évaluation des performances avec des graphiques
4. Prédictions sur de nouveaux fichiers audio


In [ ]:

# Importation des bibliothèques nécessaires
import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix


## Préparation des données

### Génération des Spectrogrammes

In [ ]:

# Générer des spectrogrammes à partir de fichiers audio
audio_folder = 'song'  # Chemin vers les fichiers audio
output_folder = 'dataset'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for file_name in os.listdir(audio_folder):
    if file_name.endswith(('.wav', '.mp3')):
        audio_path = os.path.join(audio_folder, file_name)
        y, sr = librosa.load(audio_path, sr=None)
        S = np.abs(librosa.stft(y))
        plt.figure(figsize=(10, 6))
        librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max), sr=sr, y_axis='log', x_axis='time')
        plt.colorbar(format='%+2.0f dB')
        plt.title(file_name)
        plt.savefig(f"{output_folder}/{os.path.splitext(file_name)[0]}_spectrogram.png")
        plt.close()


![Spectograme](dataset/C3/c3-95007_spectrogramme.png)

## Dataset
```plaintext
├── dataset/
│   ├── C/
│   │   ├── spectrogram1.png
│   │   ├── spectrogram2.png
│   ├── D/
│   ├── ...
```

## Chargement et Prétraitement des Données

In [ ]:
# Étape 2 : Chargement des données
# Utilisation de ImageDataGenerator pour charger les spectrogrammes
train_datagen = ImageDataGenerator(rescale=1./255)  # Normalisation des images
train_generator = train_datagen.flow_from_directory(
    'dataset/',  # Répertoire contenant les sous-dossiers de notes
    target_size=(256, 256),  # Taille des images
    batch_size=32,
    class_mode='categorical'  # Utilisation de la classification catégorielle
)

```plaintext

├── Chargement des données/
    ├── ImageDataGenerator/
        ├── Applique une normalisation des images avec `rescale=1./255`.
        ├── Convertit les pixels des images de 0-255 à une plage de 0-1.
        ├── Objectif : Faciliter le traitement des données par le modèle et stabiliser l'entraînement.
    ├── flow_from_directory/
        ├── Charge automatiquement les images depuis le dossier `dataset/`.
        ├── Organise les données en fonction des sous-dossiers correspondant aux classes (par exemple, A, B, C...).
        ├── Redimensionne les images à une taille fixe de `(256, 256)` pour qu'elles soient compatibles avec l'entrée du modèle.
        ├── Divise les données en **lots de 32 images** grâce au paramètre `batch_size=32`, ce qui optimise la mémoire.
        ├── Définit la tâche comme une **classification multi-classes** avec `class_mode='categorical'`.
            ├── Convertit les étiquettes des classes en vecteurs "one-hot" (exemple : A = [1, 0, 0, 0], B = [0, 1, 0, 0], etc.).
```

## Création du Modèle

In [ ]:

    # Création et compilation du modèle
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


### Les couches (6 couches principales)
```plaintext
├── CNN/
│   ├── Couches de traitement des motifs/
│      ├── 2 couches Conv2D (convolutionnelles)
│      ├── 2 couches MaxPooling2D (réduction de taille)
│   ├── Couches de classification/
│      ├── 1 couche Flatten (aplatissement des données)
│      ├── 2 couches Dense (neurones entièrement connectés)
```

### Plus en detail
```plaintext
├── CNN/
    ├── Première couche Conv2D/
        ├── Applique 32 filtres de taille (3x3) sur le spectrogramme.
        ├── Détecte des motifs locaux simples.
        ├── Rend le spectrogramme plus riche en motifs distinctifs.
    ├── Première couche MaxPooling2D/
        ├── Réduit la taille du spectrogramme en ne conservant que les maxima locaux (2x2).
        ├── Diminue les détails inutiles tout en gardant les informations essentielles.
        ├── Effet : Compresse les données tout en préservant les zones clés.
    ├── Deuxième couche Conv2D/
        ├── Applique 64 filtres de taille (3x3) pour capturer des motifs plus complexes.
        ├── Identifie des relations entre les motifs simples trouvés par la première couche.
        ├── Exemple : Repérer des formes ou textures sur le spectrogramme.
    ├── Deuxième couche MaxPooling2D/
        ├── Réduit encore la taille du spectrogramme (2x2).
        ├── Diminue la complexité spatiale tout en gardant l'essentiel.
        ├── Effet : Concentre les informations importantes sur des régions spécifiques.
    ├── Flatten/
        ├── Transforme le spectrogramme compressé en une grande liste linéaire.
        ├── Effet : Prépare les données pour entrer dans les couches denses.
    ├── Couches Dense/
        ├── Dense (128 neurones) :
            ├── Combine les motifs extraits en apprenant des relations complexes.
            ├── Prépare des représentations plus abstraites pour la classification.
        ├── Dense (sortie) :
            ├── Applique la fonction softmax pour convertir les sorties en probabilités.
            ├── Donne une probabilité pour chaque classe (note musicale prédite).

```

## Entraînement du Modèle

In [ ]:

# Entraîner le modèle
model.fit(train_generator, validation_data=validation_generator, epochs=10)

# Sauvegarder le modèle
model.save('mon_modele.h5')


## Visualisation des Performances

In [ ]:

# Courbes de perte et de précision
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Perte Entraînement')
plt.plot(history.history['val_loss'], label='Perte Validation')
plt.title('Courbe de Perte')
plt.xlabel('Époque')
plt.ylabel('Perte')
plt.legend()
plt.savefig('courbe_perte.png')
plt.close()

plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Précision Entraînement')
plt.plot(history.history['val_accuracy'], label='Précision Validation')
plt.title('Courbe de Précision')
plt.xlabel('Époque')
plt.ylabel('Précision')
plt.legend()
plt.savefig('courbe_precision.png')
plt.close()


## Matrice de Confusion

![Spectograme](confusion.png)

## Courbe de Perte

![Spectograme](perte.png)

## Courbe de precision

![Spectograme](precision.png)

## Prédiction sur un Nouvel audio

In [ ]:

# Charger un modèle pré-entrainé et prédire une note
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

model = load_model('mon_modele.h5')

audio_path = 'cnote.mp3'
y, sr = librosa.load(audio_path, sr=None)
S = np.abs(librosa.stft(y))
plt.figure(figsize=(10, 6))
librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max), sr=sr, y_axis='log', x_axis='time')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogramme')
plt.savefig("spectrogramme_test.png")
plt.close()

img_path = 'spectrogramme_test.png'
img = image.load_img(img_path, target_size=(256, 256))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)

class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
predicted_note = class_names[predicted_class[0]]

print(f"La note prédite est : {predicted_note}")


## Résultat 
La note prédite est : G

## Conclusion


### Résultats
- Le modèle est entraîné avec une précision de validation acceptable.
- Les spectrogrammes permettent une bonne classification des notes.

### Améliorations futures
- Ne pas se limiter au piano
- Prediction de plus de note 
- Conversion audio complet en midi 

